# Codificació - Modulació

In [32]:
import numpy as np
import random
import binascii

In [91]:
llista_a = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
Menc = np.array([[6, 24, 1],
                 [13, 16, 10],
                 [20, 17, 15]])
Mdes = np.array([[8, 5, 10],
                 [21, 8, 21],
                 [21, 12, 8]])

# TODO: Gen random string sequence

def transforma_text(string):
    text = list(string.lower()) # Transform string into a char list (lowercase)
    for c in text:
        if c in llista_a: # If the char pertains to the alphabet, swap it for its corresponding index
            text[text.index(c)] = llista_a.index(c)
    return text

def encripta(idx_list):
    code_list = []
    #counter = 0
    #guardar posicio caracters
    for i in np.arange(0, len(idx_list), 3):
        # Filter NaNs
        vector = np.array([idx_list[i], idx_list[i+1], idx_list[i+2]]) # If missing, add zeros!!!
        aux = (Menc.dot(vector))%26 #multiply%26
        #print(aux)
        code_list += aux.tolist()
        #add new vector to code
    return code_list

# This gives us the equivalent of the codification in NRZ
def binarize(code_list):
    binary_list = []
    for i in code_list:
        aux = str(i).zfill(2) # If number < 10, we add a 0 to its left
        #print(aux)
        for j in str(i):
            binary_list += bin(ord(j))[2:].zfill(8)
    return binary_list

In [92]:
### CODIFICATION METHODS ###

# NRZ
def encode_NRZ(binary_list):
    #return map(lambda x: 1 if x else -1, binary_list)
    return ['1' if x != '0' else '-1' for x in binary_list]
    #return [x for x in binary_list else -1]
    
# NRZ-L
def encode_NRZL():
    return ['-1' if x != '0' else '1' for x in binary_list]

In [93]:
#test = transforma_text("Ola k ase, etc.")
test = transforma_text("testabcok")
print(test)
result = encripta(test)
print(result)
binlist = binarize(result)
#binlist = string2bits(result)
print(binlist)
NRZ_list = encode_NRZ(binlist)
print(NRZ_list)

[19, 4, 18, 19, 0, 1, 2, 14, 10]
[20, 23, 16, 11, 23, 5, 20, 12, 12]
['0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0']
['-1', '-1', '1', '1', '-1', '-1', '1', '-1', '-1', '-1', '1', '1', '-1', '-1', '-1', '-1', '-1', '-1', '1', '1', '-1', '-1', '1', '-1', '-1', '-1', '1', '1', '-1', '-1', '1', '1', '-1', '-1', '1', '1', '-1', '-1', '-1', '1', '-1', '-1', '1', '1', '-